In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
SHAPE = None
NREALS = 500
NFREQS = 40
NLOUDEST = 10
BGL = 1

TOL=0.01
MAXBADS=5

NVARS = 21

NPSRS = 40
NSKIES = 100

ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

In [ ]:
target='gsmf_phi0'

white_noise, count_cws = detstats.build_noise_arrays(target)

In [ ]:
ylabel='N(DP_SS>0.5)'

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', yscale='linear', height=7)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    xx, _ = detstats.get_ratio_arrays(target)
    _, count_cws = detstats.build_noise_arrays(target)

    plot.draw_med_conf(ax, xx, yy)